# SQL query from table names - Continued

In [10]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [11]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    { $
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [12]:
context = [
    {'role': 'system', 'content': """
### Database Schema:
You are tasked with generating SQL queries. The following schema describes the database:

CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(50)
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level VARCHAR(50),
    institution VARCHAR(100),
    years DATE,
    speciality VARCHAR(50),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

### Instructions:
1. Generate valid SQL queries using the schema above.
2. Always use SQL syntax compatible with SQLite.
3. If a query cannot be answered based on the schema, respond with: "I cannot answer this query based on the provided schema."

### Examples:

Q: "List all employees and their names."
SQL: SELECT ID_usr, name FROM employees;

Q: "What is the average salary for each year?"
SQL: SELECT year, AVG(salary) AS average_salary FROM salary GROUP BY year;

Q: "Find employees who studied at 'MIT'."
SQL: SELECT e.name, s.institution 
     FROM employees e
     JOIN studies s ON e.ID_usr = s.ID_usr
     WHERE s.institution = 'MIT';

Q: "Show all employees and their salaries for 2023."
SQL: SELECT e.name, s.salary 
     FROM employees e
     JOIN salary s ON e.ID_usr = s.ID_usr
     WHERE s.year = '2023';

Now, provide the SQL query for the given user request.
"""}
]

In [13]:
#FEW SHOT SAMPLES
context.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite,
-- answer the following questions for the table provided above.

### Table Definitions:
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(50)
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level VARCHAR(50),
    institution VARCHAR(100),
    years DATE,
    speciality VARCHAR(50),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

### Few-Shot Examples:

Q: "List all employees and their names."
SQL: SELECT ID_usr, name FROM employees;

Q: "Find the average salary for each year."
SQL: SELECT year, AVG(salary) AS average_salary FROM salary GROUP BY year;

Q: "Which employees have studied at 'Harvard University'?"
SQL: SELECT e.name, s.institution
     FROM employees e
     JOIN studies s ON e.ID_usr = s.ID_usr
     WHERE s.institution = 'Harvard University';

Q: "Show all employees and their salaries for 2023."
SQL: SELECT e.name, s.salary 
     FROM employees e
     JOIN salary s ON e.ID_usr = s.ID_usr
     WHERE s.year = '2023';

Q: "How many employees are in the database?"
SQL: SELECT COUNT(*) AS total_employees FROM employees;

Q: "List employees with their salaries and educational levels."
SQL: SELECT e.name, s.salary, st.educational_level
     FROM employees e
     LEFT JOIN salary s ON e.ID_usr = s.ID_usr
     LEFT JOIN studies st ON e.ID_usr = st.ID_usr;

-- Now answer the user's question based on the provided tables and examples above.
"""})

In [14]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [19]:
#new
context_user = context.copy()
print(return_CCRMSQL("Find employees whose salaries are greater than 50,000 in 2023.", context_user))

SQL: 
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 50000 AND s.year = '2023';


In [24]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("What is the average salary of employees who studied at ‘MIT’?", old_context_user))

This is your SQL:
```sql
SELECT AVG(s.salary) AS average_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE st.Institution = 'MIT';
```

This SQL query retrieves the average salary of employees who studied at 'MIT' by joining the employees, studies, and salary tables on the employee ID. It then filters the results to only include employees who studied at 'MIT' and calculates the average salary of those employees.


In [23]:
#new
print(return_CCRMSQL("What is the average salary of employees who studied at ‘MIT’?’", context_user))

SQL: 
SELECT AVG(s.salary) AS average_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE st.institution = 'MIT';


In [22]:
#old
print(return_CCRMSQL("Find employees who earned a salary of more than 60,000 in 2022 and studied ‘Computer Science’", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE s.salary > 60000 AND s.year = '2022' AND st.Speciality = 'Computer Science';
```

This SQL query retrieves the names of employees who earned a salary of more than 60,000 in the year 2022 and studied 'Computer Science'. It achieves this by joining the three tables (employees, salary, and studies) on their corresponding ID columns and then filtering the results based on the specified conditions.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?